In [2]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()


,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Drop unnecessary columns
application_df = application_df.drop(columns=["EIN", "NAME"])

# Group rare categories for APPLICATION_TYPE
application_type_counts = application_df['APPLICATION_TYPE'].value_counts()
rare_application_types = application_type_counts[application_type_counts < 100].index
application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(rare_application_types, "Other")

# Group rare categories for CLASSIFICATION
classification_counts = application_df['CLASSIFICATION'].value_counts()
rare_classifications = classification_counts[classification_counts < 100].index
application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(rare_classifications, "Other")

# One-hot encode categorical variables
application_df = pd.get_dummies(application_df, drop_first=True)

# Split the data into features and target
X = application_df.drop(columns=["IS_SUCCESSFUL"])
y = application_df["IS_SUCCESSFUL"]

# Split into training and testing datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [4]:
# Define the neural network model
nn = tf.keras.models.Sequential()

# Input and hidden layers
nn.add(tf.keras.layers.Dense(units=100, activation="relu", input_dim=X_train.shape[1]))
nn.add(tf.keras.layers.Dense(units=50, activation="tanh"))
nn.add(tf.keras.layers.Dense(units=25, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the model
nn.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])


c:\Users\TeddySong\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [5]:
from tensorflow.keras.callbacks import EarlyStopping

# Add early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
history = nn.fit(X_train_scaled, y_train, epochs=100, batch_size=32, validation_data=(X_test_scaled, y_test), callbacks=[early_stopping])


Epoch 1/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.7057 - loss: 0.5794 - val_accuracy: 0.7219 - val_loss: 0.5592
Epoch 2/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7251 - loss: 0.5533 - val_accuracy: 0.7236 - val_loss: 0.5565
Epoch 3/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7290 - loss: 0.5516 - val_accuracy: 0.7204 - val_loss: 0.5654
Epoch 4/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 994us/step - accuracy: 0.7289 - loss: 0.5514 - val_accuracy: 0.7248 - val_loss: 0.5580
Epoch 5/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7313 - loss: 0.5507 - val_accuracy: 0.7259 - val_loss: 0.5530
Epoch 6/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7265 - loss: 0.5516 - val_accuracy: 0.7296 - val_loss: 0.5566
Epoch 7/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7324 - loss: 0.5478 - val_accuracy: 0.7242 - val_loss: 0.5539
Epoch 8/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 965us/step - accuracy: 0.7327 - loss: 0.5459 - val_

In [6]:
# Evaluate the model
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

# Save the optimized model
nn.save("AlphabetSoupCharity_Optimization.h5")


215/215 - 0s - 777us/step - accuracy: 0.7271 - loss: 0.5521


Loss: 0.5521393418312073, Accuracy: 0.7271137237548828


In [10]:
import tensorflow as tf

# Define the optimized model
nn = tf.keras.models.Sequential()

# Input layer and first hidden layer
nn.add(tf.keras.layers.Dense(units=150, activation="relu", input_dim=X_train.shape[1]))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=80, activation="relu"))

# Dropout layer
nn.add(tf.keras.layers.Dropout(0.2))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=50, activation="tanh"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the model
nn.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])


In [11]:
from tensorflow.keras.callbacks import EarlyStopping

# Add early stopping callback
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model
history = nn.fit(
    X_train_scaled, y_train,
    epochs=100, batch_size=16,
    validation_data=(X_test_scaled, y_test),
    callbacks=[early_stopping]
)



Epoch 1/100
1715/1715 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.7099 - loss: 0.5805 - val_accuracy: 0.7236 - val_loss: 0.5605
Epoch 2/100
1715/1715 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.7290 - loss: 0.5532 - val_accuracy: 0.7261 - val_loss: 0.5635
Epoch 3/100
1715/1715 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.7323 - loss: 0.5495 - val_accuracy: 0.7245 - val_loss: 0.5548
Epoch 4/100
1715/1715 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.7272 - loss: 0.5547 - val_accuracy: 0.7206 - val_loss: 0.5564
Epoch 5/100
1715/1715 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.7292 - loss: 0.5530 - val_accuracy: 0.7300 - val_loss: 0.5569
Epoch 6/100
1715/1715 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.7317 - loss: 0.5517 - val_accuracy: 0.7302 - val_loss: 0.5568
Epoch 7/100
1715/1715 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.7302 - loss: 0.5480 - val_accuracy: 0.7270 - val_loss: 0.5569
Epoch 8/100
1715/1715 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.7315 - loss: 0

In [12]:
# Evaluate the model
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

# Save the model
nn.save("AlphabetSoupCharity_Optimization.h5")


215/215 - 0s - 782us/step - accuracy: 0.7283 - loss: 0.5533


Loss: 0.5532643795013428, Accuracy: 0.7282798886299133


In [13]:
# Define the optimized neural network model
nn = tf.keras.models.Sequential()

# Input layer and first hidden layer
nn.add(tf.keras.layers.Dense(units=150, activation="relu", input_dim=X_train.shape[1]))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=80, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=50, activation="tanh"))

# Add a dropout layer to prevent overfitting
nn.add(tf.keras.layers.Dropout(0.2))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the model
nn.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

# Add early stopping to prevent overtraining
early_stopping = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)

# Train the model
history = nn.fit(
    X_train_scaled, y_train,
    epochs=100, batch_size=32,
    validation_data=(X_test_scaled, y_test),
    callbacks=[early_stopping]
)

# Evaluate the model
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

# Save the optimized model
nn.save("AlphabetSoupCharity_Optimization.keras")

Epoch 1/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.7094 - loss: 0.5844 - val_accuracy: 0.7286 - val_loss: 0.5620
Epoch 2/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7279 - loss: 0.5585 - val_accuracy: 0.7232 - val_loss: 0.5595
Epoch 3/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7291 - loss: 0.5520 - val_accuracy: 0.7270 - val_loss: 0.5544
Epoch 4/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7331 - loss: 0.5488 - val_accuracy: 0.7267 - val_loss: 0.5530
Epoch 5/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7290 - loss: 0.5498 - val_accuracy: 0.7308 - val_loss: 0.5531
Epoch 6/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7297 - loss: 0.5508 - val_accuracy: 0.7265 - val_loss: 0.5547
Epoch 7/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7355 - loss: 0.5425 - val_accuracy: 0.7294 - val_loss: 0.5526
Epoch 8/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7321 - loss: 0.5445 - val_accu